# Intelligent Systems Assignment 1

## Masterball solver

Nicolás Esteban Balcero Cerón - CC 1032452083



### 1. Create a class to model the Masterball problem

A Masterball must be represented as an array of arrays with integer values representing the color of the tile in each position:

A solved masterball must look like this:

```python
[ [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7]
]
```

#### Variables modeling the actions

In [1]:
'''
This variables MUST not be changed.
They represent the movements of the masterball.
'''
R_0 = "Right 0"
R_1 = "Right 1"
R_2 = "Right 2"
R_3 = "Right 3"

V_0 = "Vertical 0"
V_1 = "Vertical 1"
V_2 = "Vertical 2"
V_3 = "Vertical 3"
V_4 = "Vertical 4"
V_5 = "Vertical 5"
V_6 = "Vertical 6"
V_7 = "Vertical 7"

actions = [R_0,R_1,R_2,R_3,V_0,V_1,V_2,V_3,V_4,V_5,V_6,V_7]


`R_i` moves the `i`th row to the right. For instance, `R_2` applied to the solved state will produce:

```python
[ [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7],
  [7, 0, 1, 2, 3, 4, 5, 6],
  [0, 1, 2, 3, 4, 5, 6, 7]
]
```

`V_i` performs a clockwise vertical move starting with the `i`th column

`V_1` applied to the above state will produce:

```python
[ [0, 4, 3, 2, 1, 5, 6, 7],
  [0, 3, 2, 1, 0, 5, 6, 7],
  [7, 4, 3, 2, 1, 4, 5, 6],
  [0, 4, 3, 2, 1, 5, 6, 7]
]
```

#### The Masterball problem class

In [2]:
import search
import util

class MasterballProblem(search.SearchProblem):    
    
    def __init__(self, startState):
        '''
        Store the initial state in the problem representation and any useful
        data.
        Here are some examples of initial states:
        [[0, 1, 4, 5, 6, 2, 3, 7], [0, 1, 3, 4, 5, 6, 3, 7], [1, 2, 4, 5, 6, 2, 7, 0], [0, 1, 4, 5, 6, 2, 3, 7]]
        [[0, 7, 4, 5, 1, 6, 2, 3], [0, 7, 4, 5, 0, 5, 2, 3], [7, 6, 3, 4, 1, 6, 1, 2], [0, 7, 4, 5, 1, 6, 2, 3]]
        [[0, 1, 6, 4, 5, 2, 3, 7], [0, 2, 6, 5, 1, 3, 4, 7], [0, 2, 6, 5, 1, 3, 4, 7], [0, 5, 6, 4, 1, 2, 3, 7]]
        '''
        self.expanded = 0
        ### your code here ###
        self.startState = startState
    
    
    def isGoalState(self, state):
        '''
        Define when a given state is a goal state (A correctly colored masterball)
        '''
        ### your code here ###
        for idx, lat in enumerate(state):
            i_vector = list(lat)
            for i in range(1,len(i_vector)):
                if i != i_vector[i]:
                    return False
        return True
    
    def getStartState(self):
        '''
        Implement a method that returns the start state according to the SearchProblem
        contract.
        '''
        ### your code here ###
        return self.startState

    def getSuccessors(self, state):
        '''
        Implement a successor function: Given a state from the masterball
        return a list of the successors and their corresponding actions. 

        This method *must* return a list where each element is a tuple of 
        three elements with the state of the masterball in the first position,
        the action (according to the definition above) in the second position, 
        and the cost of the action in the last position. 

        Note that you should not modify the state.
        '''
        self.expanded += 1
        successors = []
        ### your code here ###
        s_state = []
        for layer in state:
            s_state.append(list(layer))

        for action in actions:
            s_state = []
            for layer in state:
                s_state.append(list(layer))

            action_type, orientation = action.split(" ")
            orientation = int(orientation)
            if action_type == "Right":
                s_state[orientation] = [s_state[orientation][-1]] + s_state[orientation][:-1]
            elif action_type == "Vertical":
                lat = 0
                while lat < len(s_state):
                    aux_arr = [s_state[lat][(orientation+3)%8],s_state[lat][(orientation+2)%8],s_state[lat][(orientation+1)%8],s_state[lat][orientation]]
                    s_state[lat][orientation] = aux_arr[0]
                    s_state[lat][(orientation+1)%8] = aux_arr[1]
                    s_state[lat][(orientation+2)%8] = aux_arr[2]
                    s_state[lat][(orientation+3)%8] = aux_arr[3]
                    lat += 1


            tupled_layers = []
            for layer in s_state:
                tupled_layers.append(tuple(layer))

            tupled_state = tuple(tupled_layers)

            successors.append(tuple((tupled_state,action,1)))
        return successors

### 2. Implement iterative deepening search

Follow the example code provided in class and implement iterative deepening search (IDS).

In [3]:
from graphviz import Graph, Digraph
from IPython.display import display
import itertools

class search_tree():
    def __init__(self):
        self.graph = Digraph(graph_attr = {'size':'9'})

    def addNode(self, name, label):
        self.graph.node(name, label)

    def addEdge(self, source, action, target):
        self.graph.edge(source, target, action)

    def getDot(self):
        return self.graph

def graphDot(g_prob, color):
    dot = Graph(graph_attr = {'size':'3.5'})
    for node in g_prob.G:
        if not node in color:
            dot.node(node)
        else:
            dot.node(node, style = 'filled', color = color[node])
    for n1 in g_prob.G:
        for n2 in g_prob.G[n1]:
            if n1 < n2:
                dot.edge(n1, n2)
    return dot

def iterativeDeepeningSearch(problem):    
    for depth in itertools.count():
        visited = {}
        frontier = util.Queue()
        state = problem.getStartState()
        frontier.push((state, [], 0))
        tree = search_tree()
        tree.addNode(str(state)+"[]",str(state))
        while not frontier.isEmpty():
            u, actions, path_cost = frontier.pop()
            if problem.isGoalState(u):
                return  actions, tree
            if not u in visited:
                for v, action, cost in problem.getSuccessors(u):
                    tree.addNode(str(v) + str(actions+[action]), str(v))
                    tree.addEdge(str(u) + str(actions), str(cost), str(v) + str(actions+[action]))
                    if path_cost < depth:
                        frontier.push((v, actions + [action], path_cost + cost))
            visited[u] = 'black'
            display(tree)
    return [], tree

def aStarSearch(problem, heuristic):
    def f_cost(item):
        return item[2] + heuristic(item[0])
    visited = {}
    frontier = util.PriorityQueueWithFunction(f_cost)
    state = problem.getStartState()
    frontier.push((state, [], 0))
    tree = search_tree()
    tree.addNode(str(state)+"[]",str(state))
    while not frontier.isEmpty():
        u, actions, path_cost = frontier.pop()
        if problem.isGoalState(u):
            return  actions, tree
        if not u in visited:
            for v, action, cost in problem.getSuccessors(u):
                tree.addNode(str(v) + str(actions+[action]), str(v))
                tree.addEdge(str(u) + str(actions), str(cost), str(v) + str(actions+[action]))
                frontier.push((v, actions + [action], path_cost + cost))
        visited[u] = 'black'
    return [], tree

Evaluate it to see what is the maximum depth that it could explore in a reasonable time. Report the results. 

### 3. Implement different heuristics for the problem

Implement at least two admissible and consistent heuristics. Compare A* using the heuristics against IDS calculating the number of expanded nodes and the effective branching factor, in the same way as it is done in figure 3.29 of [Russell10].

In [4]:
# def myHeuristic(state):
#     base_vector = []
#     i_0 = None
#     error_count = 0
#     for idx, lat in enumerate(state):
#         i_vector = list(lat)
#         if idx == 0:
#             base_vector = list(lat)
#             try:
#                 i_0 = base_vector.index(0)
#             except:
#                 error_count += 2
#         for i in range(1,len(i_vector)):
#             if i != i_vector[(i_0+i)%8]:
#                 error_count += 1
#     return error_count
def myHeuristic(state):
    ### your code here ###
    error_count = 0
    for lat in state:
        i = 0
        for ipos in lat:
            if ipos != i:
                error_count+=1
            i+=1
    return error_count

Now apply the develop functions to solve the Masterball problem.

In [ ]:
#problem = MasterballProblem(((0, 1, 4, 5, 6, 2, 3, 7), (0, 1, 3, 4, 5, 6, 3, 7), (1, 2, 4, 5, 6, 2, 7, 0), (0, 1, 4, 5, 6, 2, 3, 7)))
#s = aStarSearch(problem, myHeuristic)
#print(s)
#display(s[1].getDot())

#s = iterativeDeepeningSearch(problem, myHeuristic)
#print(s)
#display(s[1].getDot())

import random
from test import *
import os
import time

solved_problem = MasterballProblem(( (0, 1, 2, 3, 4, 5, 6, 7),
                              (0, 1, 2, 3, 4, 5, 6, 7),
                              (0, 1, 2, 3, 4, 5, 6, 7),
                              (0, 1, 2, 3, 4, 5, 6, 7)))

action_limit = 1
required_states = 5

generated_states = []
for s in range(required_states):
    r_i = random.randint(0,len(actions))
    i_state = solved_problem.getSuccessors(solved_problem.getStartState())[r_i-1]
    for i in range(action_limit):
        r_i = random.randint(0,len(actions))
        i_state = solved_problem.getSuccessors(list(i_state)[0])[r_i-1]

    generated_states.append(i_state)
    print(i_state)
    
for s in generated_states:
    print("SOLVING")
    print(s[0])
    problem = MasterballProblem(s[0])
    a_time = time.time()
    a, t = aStarSearch(problem, myHeuristic)
    a_time =  time.time() - a_time
    d_time = time.time()
    b, u = iterativeDeepeningSearch(problem)
    d_time =  time.time() - d_time
    print(str(a) + " " + str(a_time) + " " + str(problem.getStartState()))
    display(t.getDot())
    print(str(b) + " " + str(d_time) + " " + str(problem.getStartState()))
    display(u.getDot())


(((3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6)), 'Vertical 0', 1)
(((0, 1, 2, 6, 5, 4, 3, 7), (0, 1, 2, 5, 4, 3, 2, 7), (7, 0, 1, 6, 5, 4, 3, 6), (0, 1, 2, 6, 5, 4, 3, 7)), 'Vertical 3', 1)
(((0, 1, 2, 6, 5, 4, 3, 7), (7, 0, 1, 6, 5, 4, 3, 6), (0, 1, 2, 5, 4, 3, 2, 7), (0, 1, 2, 6, 5, 4, 3, 7)), 'Vertical 3', 1)
(((1, 4, 3, 7, 0, 5, 6, 2), (1, 4, 3, 7, 0, 5, 6, 2), (1, 4, 3, 7, 0, 5, 6, 2), (1, 4, 3, 7, 0, 5, 6, 2)), 'Vertical 1', 1)
(((0, 1, 5, 6, 2, 3, 4, 7), (0, 1, 5, 6, 2, 3, 4, 7), (0, 1, 5, 6, 2, 3, 4, 7), (0, 1, 5, 6, 2, 3, 4, 7)), 'Vertical 3', 1)
SOLVING
((3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6))
['Vertical 0', 'Vertical 5'] 0.00182199478149 ((3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6))


['Vertical 0', 'Vertical 5'] 0.0343601703644 ((3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6), (3, 2, 1, 5, 4, 0, 7, 6))


SOLVING
((0, 1, 2, 6, 5, 4, 3, 7), (0, 1, 2, 5, 4, 3, 2, 7), (7, 0, 1, 6, 5, 4, 3, 6), (0, 1, 2, 6, 5, 4, 3, 7))
['Vertical 3', 'Right 2', 'Right 2', 'Right 2', 'Right 2', 'Right 2', 'Right 2', 'Right 2'] 0.0192220211029 ((0, 1, 2, 6, 5, 4, 3, 7), (0, 1, 2, 5, 4, 3, 2, 7), (7, 0, 1, 6, 5, 4, 3, 6), (0, 1, 2, 6, 5, 4, 3, 7))


['Vertical 7', 'Right 1', 'Vertical 3', 'Vertical 7'] 3.362885952 ((0, 1, 2, 6, 5, 4, 3, 7), (0, 1, 2, 5, 4, 3, 2, 7), (7, 0, 1, 6, 5, 4, 3, 6), (0, 1, 2, 6, 5, 4, 3, 7))


# 

In [ ]:
#print(iterativeDeepeningSearch(problem))
s = aStarSearch(problem, myHeuristic)
print(s)
display(s[1].getDot())

